In [1]:
import requests as rq
import lxml.html as lx
import pandas as pd
import re

In [2]:
#read html + make df for general stats


url = 'https://www.espn.com/nba/team/stats/_/name/sac/salary'
names = pd.read_html(url)[0]
stats = pd.read_html(url)[1]
kings_stats = pd.concat([names,stats],axis = "columns")

In [3]:
# read html + make df for shooting stats


shoot = pd.read_html(url)[3]
kings_shoot_stats = pd.concat([names,shoot], axis = "columns")

In [4]:
# clean up "name" and drop total from df


words = []
for i in kings_shoot_stats["Name"]:
    words.append(i.split()[0:2])
words = pd.DataFrame(words)
words.columns = ["First","Last"]
words["Names"] = words['First']+" "+ words["Last"]
words = words.drop("First", axis = 1).drop("Last", axis = 1)
kings_shoot_stats["Name"] = words
kings_shoot_stats = kings_shoot_stats.drop(16)

In [5]:
#create salary df
url2 = "https://www.espn.com/nba/team/roster/_/name/sac/salary"
kings_salary_raw = pd.read_html(url2)[0]

In [6]:
#Strips player number from each players name

for i in range(0,len(kings_salary_raw)):
    kings_salary_raw["Name"][i] = re.sub(r"\d+", "", str(kings_salary_raw["Name"][i]))


C:\Users\Jonathan\AppData\Local\Temp\ipykernel_6456\3064642318.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kings_salary_raw["Name"][i] = re.sub(r"\d+", "", str(kings_salary_raw["Name"][i]))


In [7]:
#Cleans dataframe to only player name + salary
kings_salary = kings_salary_raw.set_index("Name").drop('Unnamed: 0', axis = 1)
kings_salary.columns
kings_salary = kings_salary.drop(['POS', 'Age', 'HT', 'WT', 'College'], axis = 1)

In [8]:
#combine dfs. final kings 2022 df


kings_stats_22 = kings_shoot_stats.join(kings_salary, on = "Name")

In [9]:
nbateamstats = pd.DataFrame()
for i in range(2023,2018, -1):
    tempurl = "https://www.espn.com/nba/stats/team/_/view/opponent/season/"+str(i)+"/seasontype/2"
    team_names = pd.read_html(tempurl)[0]
    team_stats = pd.read_html(tempurl)[1]
    team_stats
    team_concat = pd.concat([team_names,team_stats],axis = "columns")
    team_concat['Team'] = str(i) + " " + team_concat['Team'].astype(str)
    nbateamstats = pd.concat([nbateamstats, team_concat], ignore_index = True)
nbateamstats



,RK,Team,GP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,FTA,FT%,OR,DR,REB,AST,STL,BLK,TO,PF
0,1,2023 Dallas Mavericks,16,105.3,38.3,81.2,47.1,10.1,28.3,35.8,...,23.3,80.2,9.3,32.8,42.1,22.5,6.3,5.2,15.3,23.3
1,2,2023 LA Clippers,18,105.8,39.2,88.5,44.3,11.8,35.1,33.8,...,20.9,75.0,10.8,32.5,43.3,22.8,8.3,4.1,13.3,18.9
2,3,2023 Philadelphia 76ers,16,106.6,39.1,84.6,46.2,10.4,32.7,31.7,...,23.3,77.7,9.9,33.9,43.8,23.8,6.4,4.8,14.4,20.1
3,4,2023 Milwaukee Bucks,16,106.9,40.0,89.9,44.5,11.0,31.1,35.3,...,20.3,78.4,10.1,33.7,43.8,21.3,6.3,5.1,13.6,20.8
4,5,2023 Phoenix Suns,16,107.9,38.4,84.1,45.7,11.9,31.1,38.4,...,24.2,79.6,10.5,31.4,41.9,23.1,6.0,4.1,14.9,20.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,26,2019 Sacramento Kings,82,115.3,42.3,90.6,46.6,12.0,34.6,34.8,...,24.4,77.0,11.2,36.8,48.0,24.7,7.5,5.1,15.3,21.4
146,27,2019 New Orleans Pelicans,82,116.8,43.3,92.8,46.6,12.3,33.7,36.4,...,23.5,76.4,11.0,34.8,45.8,26.5,8.6,5.4,13.1,21.0
147,28,2019 Phoenix Suns,82,116.8,42.5,88.2,48.2,11.1,29.9,37.2,...,27.6,75.3,11.8,35.3,47.2,25.9,9.0,5.0,15.2,20.7
148,29,2019 Washington Wizards,82,116.9,43.4,90.5,48.0,12.1,32.8,37.0,...,23.1,77.8,11.4,35.8,47.3,26.2,7.7,4.6,15.0,21.2


In [10]:
#nbateams = tab_dict['tab_2021']["Team"]
nbateams = pd.read_html("https://www.espn.com/nba/stats/team/_/view/opponent/season/2023/seasontype/2")[0]["Team"]

In [11]:
def clean_teams(team, stats):
    team1 = team.columns[0]
    team.columns = ["Team"]
    new_row = pd.DataFrame({team.columns[0]:team1}, index=[0])
    team = pd.concat([new_row,team.loc[:]]).reset_index(drop=True)
    for i in range(0, len(team.iloc[:,0])):
        for teamname in nbateams:
            if team.iloc[:,0][i].__contains__(teamname):
                team.iloc[:,0][i] = teamname
                break
            else:
                continue
    result = team.join(stats.iloc[:,0:2])
    return(result)
    
def add_playoff_status(df, year):
    if year == 2023:
        df["Made Playoffs"] = None
        return(df)
    df["Made Playoffs"] = 0
    if year in [2019, 2020]:
        for index in df.index:
            if index < 8:
                df.iloc[index,3] = 1
    else:
        for index in df.index:
            if index < 10:
                df.iloc[index,3] = 1
    return(df)


league_wins_losses = pd.DataFrame()
for i in range(2023,2018, -1):
    urlstandings = "https://www.espn.com/nba/standings/_/season/"+ str(i)
    teams_east = pd.read_html(urlstandings)[0]
    stats_east = pd.read_html(urlstandings)[1]
    teams_west = pd.read_html(urlstandings)[2]
    stats_west = pd.read_html(urlstandings)[3]
    teams_east = clean_teams(teams_east, stats_east)
    teams_east = add_playoff_status(teams_east, i)
    teams_west = clean_teams(teams_west, stats_west)
    teams_west = add_playoff_status(teams_west, i)
    df_win_loss = pd.concat([teams_east, teams_west], ignore_index = True)
    df_win_loss['Team'] = str(i) + " " + df_win_loss['Team'].astype(str)
    league_wins_losses = pd.concat([league_wins_losses, df_win_loss], ignore_index = True)
league_wins_losses
    

,Team,W,L,Made Playoffs
0,2023 Boston Celtics,13,4,None
1,2023 Milwaukee Bucks,12,4,None
2,2023 Cleveland Cavaliers,11,6,None
3,2023 Indiana Pacers,10,6,None
4,2023 Washington Wizards,10,7,None
...,...,...,...,...
145,2019 Minnesota Timberwolves,36,46,0
146,2019 Memphis Grizzlies,33,49,0
147,2019 New Orleans Pelicans,33,49,0
148,2019 Dallas Mavericks,33,49,0


In [12]:
fullnbastats = pd.merge(league_wins_losses, nbateamstats, on="Team")
fullnbastats = fullnbastats.drop(['RK'], axis=1)
fullnbastats

,Team,W,L,Made Playoffs,GP,PTS,FGM,FGA,FG%,3PM,...,FTA,FT%,OR,DR,REB,AST,STL,BLK,TO,PF
0,2023 Boston Celtics,13,4,None,17,113.5,42.2,91.1,46.4,11.6,...,21.9,79.6,10.2,34.8,45.0,22.3,6.2,3.9,11.7,20.5
1,2023 Milwaukee Bucks,12,4,None,16,106.9,40.0,89.9,44.5,11.0,...,20.3,78.4,10.1,33.7,43.8,21.3,6.3,5.1,13.6,20.8
2,2023 Cleveland Cavaliers,11,6,None,17,108.2,39.1,85.7,45.6,11.8,...,23.8,76.8,9.2,30.7,39.9,23.6,8.4,4.4,14.2,23.6
3,2023 Indiana Pacers,10,6,None,16,114.1,39.7,87.4,45.4,12.9,...,26.4,82.3,10.9,31.9,42.8,24.4,8.3,5.4,14.5,20.7
4,2023 Washington Wizards,10,7,None,17,109.4,39.9,89.6,44.6,12.6,...,21.8,77.8,9.8,34.0,43.8,23.5,6.8,4.2,12.1,19.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2019 Minnesota Timberwolves,36,46,0,82,114.0,42.1,90.1,46.7,12.7,...,22.3,76.7,11.2,34.4,45.6,26.2,6.6,5.5,14.3,21.9
146,2019 Memphis Grizzlies,33,49,0,82,106.1,37.6,83.4,45.1,11.6,...,24.7,78.4,9.5,35.2,44.7,23.4,7.6,4.9,14.5,21.4
147,2019 New Orleans Pelicans,33,49,0,82,116.8,43.3,92.8,46.6,12.3,...,23.5,76.4,11.0,34.8,45.8,26.5,8.6,5.4,13.1,21.0
148,2019 Dallas Mavericks,33,49,0,82,110.1,40.9,89.2,45.9,11.2,...,22.0,77.7,10.2,34.6,44.8,24.5,7.8,4.5,12.5,23.2


In [13]:
nbaefficiencystats = pd.DataFrame()
for year in range(2023,2018, -1):
    efficiencyurl = "http://www.espn.com/nba/hollinger/teamstats/_/sort/defensiveEff/year/"+ str(year)
    efficiencystats = pd.read_html(efficiencyurl)[0].iloc[1: , 1:]
    efficiencystats = pd.DataFrame(efficiencystats.values[1:], columns=efficiencystats.iloc[0])
    for i in range(0, len(efficiencystats.iloc[:,0])):
        for teamname in nbateams:
            if efficiencystats.iloc[:,0][i] == "LA Lakers":
                efficiencystats.iloc[:,0][i] = "Los Angeles Lakers"
                break
            if efficiencystats.iloc[:,0][i] in teamname:
                efficiencystats.iloc[:,0][i] = teamname
                break
            else:
                continue
    efficiencystats = efficiencystats.rename(columns = {"TEAM": "Team"})
    efficiencystats['Team'] = str(year) + " " + efficiencystats['Team'].astype(str)
    efficiencystats = efficiencystats[["Team", "PACE", "EFF FG%", "OFF EFF", "DEF EFF"]]
    nbaefficiencystats = pd.concat([nbaefficiencystats, efficiencystats], ignore_index = True)
nbaefficiencystats

1,Team,PACE,EFF FG%,OFF EFF,DEF EFF
0,2023 Milwaukee Bucks,102.1,52.0,107.0,104.0
1,2023 LA Clippers,100.4,54.9,106.0,105.0
2,2023 Cleveland Cavaliers,99.1,55.7,112.9,106.3
3,2023 Philadelphia 76ers,99.8,54.6,109.4,106.3
4,2023 New Orleans Pelicans,101.4,54.8,113.5,107.0
...,...,...,...,...,...
145,2019 Chicago Bulls,101.1,50.5,102.5,110.9
146,2019 Atlanta Hawks,106.4,52.2,105.5,111.2
147,2019 Washington Wizards,103.7,53.1,108.6,111.3
148,2019 Phoenix Suns,102.9,51.4,103.6,112.2


In [14]:
fullnbastats = pd.merge(fullnbastats, nbaefficiencystats, on="Team")
fullnbastats

,Team,W,L,Made Playoffs,GP,PTS,FGM,FGA,FG%,3PM,...,REB,AST,STL,BLK,TO,PF,PACE,EFF FG%,OFF EFF,DEF EFF
0,2023 Boston Celtics,13,4,None,17,113.5,42.2,91.1,46.4,11.6,...,45.0,22.3,6.2,3.9,11.7,20.5,101.3,57.8,116.9,110.2
1,2023 Milwaukee Bucks,12,4,None,16,106.9,40.0,89.9,44.5,11.0,...,43.8,21.3,6.3,5.1,13.6,20.8,102.1,52.0,107.0,104.0
2,2023 Cleveland Cavaliers,11,6,None,17,108.2,39.1,85.7,45.6,11.8,...,39.9,23.6,8.4,4.4,14.2,23.6,99.1,55.7,112.9,106.3
3,2023 Indiana Pacers,10,6,None,16,114.1,39.7,87.4,45.4,12.9,...,42.8,24.4,8.3,5.4,14.5,20.7,104.3,54.6,111.0,110.4
4,2023 Washington Wizards,10,7,None,17,109.4,39.9,89.6,44.6,12.6,...,43.8,23.5,6.8,4.2,12.1,19.3,100.3,53.1,107.5,107.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2019 Minnesota Timberwolves,36,46,0,82,114.0,42.1,90.1,46.7,12.7,...,45.6,26.2,6.6,5.5,14.3,21.9,102.8,51.1,108.4,110.2
146,2019 Memphis Grizzlies,33,49,0,82,106.1,37.6,83.4,45.1,11.6,...,44.7,23.4,7.6,4.9,14.5,21.4,98.8,50.8,103.8,106.2
147,2019 New Orleans Pelicans,33,49,0,82,116.8,43.3,92.8,46.6,12.3,...,45.8,26.5,8.6,5.4,13.1,21.0,105.6,52.9,108.6,110.4
148,2019 Dallas Mavericks,33,49,0,82,110.1,40.9,89.2,45.9,11.2,...,44.8,24.5,7.8,4.5,12.5,23.2,101.4,51.9,106.6,108.3


In [15]:
playoffstat = fullnbastats.pop("Made Playoffs")
fullnbastats["Made Playoffs"] = playoffstat
fullnbastats

,Team,W,L,GP,PTS,FGM,FGA,FG%,3PM,3PA,...,AST,STL,BLK,TO,PF,PACE,EFF FG%,OFF EFF,DEF EFF,Made Playoffs
0,2023 Boston Celtics,13,4,17,113.5,42.2,91.1,46.4,11.6,32.1,...,22.3,6.2,3.9,11.7,20.5,101.3,57.8,116.9,110.2,None
1,2023 Milwaukee Bucks,12,4,16,106.9,40.0,89.9,44.5,11.0,31.1,...,21.3,6.3,5.1,13.6,20.8,102.1,52.0,107.0,104.0,None
2,2023 Cleveland Cavaliers,11,6,17,108.2,39.1,85.7,45.6,11.8,33.0,...,23.6,8.4,4.4,14.2,23.6,99.1,55.7,112.9,106.3,None
3,2023 Indiana Pacers,10,6,16,114.1,39.7,87.4,45.4,12.9,34.4,...,24.4,8.3,5.4,14.5,20.7,104.3,54.6,111.0,110.4,None
4,2023 Washington Wizards,10,7,17,109.4,39.9,89.6,44.6,12.6,34.2,...,23.5,6.8,4.2,12.1,19.3,100.3,53.1,107.5,107.2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2019 Minnesota Timberwolves,36,46,82,114.0,42.1,90.1,46.7,12.7,33.6,...,26.2,6.6,5.5,14.3,21.9,102.8,51.1,108.4,110.2,0
146,2019 Memphis Grizzlies,33,49,82,106.1,37.6,83.4,45.1,11.6,32.2,...,23.4,7.6,4.9,14.5,21.4,98.8,50.8,103.8,106.2,0
147,2019 New Orleans Pelicans,33,49,82,116.8,43.3,92.8,46.6,12.3,33.7,...,26.5,8.6,5.4,13.1,21.0,105.6,52.9,108.6,110.4,0
148,2019 Dallas Mavericks,33,49,82,110.1,40.9,89.2,45.9,11.2,32.5,...,24.5,7.8,4.5,12.5,23.2,101.4,51.9,106.6,108.3,0


In [16]:
#Dataframes available:
kings_stats
kings_shoot_stats
kings_salary
nbateams
kings_stats_22
league_wins_losses
nbateamstats
nbaefficiencystats
fullnbastats

,Team,W,L,GP,PTS,FGM,FGA,FG%,3PM,3PA,...,AST,STL,BLK,TO,PF,PACE,EFF FG%,OFF EFF,DEF EFF,Made Playoffs
0,2023 Boston Celtics,13,4,17,113.5,42.2,91.1,46.4,11.6,32.1,...,22.3,6.2,3.9,11.7,20.5,101.3,57.8,116.9,110.2,None
1,2023 Milwaukee Bucks,12,4,16,106.9,40.0,89.9,44.5,11.0,31.1,...,21.3,6.3,5.1,13.6,20.8,102.1,52.0,107.0,104.0,None
2,2023 Cleveland Cavaliers,11,6,17,108.2,39.1,85.7,45.6,11.8,33.0,...,23.6,8.4,4.4,14.2,23.6,99.1,55.7,112.9,106.3,None
3,2023 Indiana Pacers,10,6,16,114.1,39.7,87.4,45.4,12.9,34.4,...,24.4,8.3,5.4,14.5,20.7,104.3,54.6,111.0,110.4,None
4,2023 Washington Wizards,10,7,17,109.4,39.9,89.6,44.6,12.6,34.2,...,23.5,6.8,4.2,12.1,19.3,100.3,53.1,107.5,107.2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2019 Minnesota Timberwolves,36,46,82,114.0,42.1,90.1,46.7,12.7,33.6,...,26.2,6.6,5.5,14.3,21.9,102.8,51.1,108.4,110.2,0
146,2019 Memphis Grizzlies,33,49,82,106.1,37.6,83.4,45.1,11.6,32.2,...,23.4,7.6,4.9,14.5,21.4,98.8,50.8,103.8,106.2,0
147,2019 New Orleans Pelicans,33,49,82,116.8,43.3,92.8,46.6,12.3,33.7,...,26.5,8.6,5.4,13.1,21.0,105.6,52.9,108.6,110.4,0
148,2019 Dallas Mavericks,33,49,82,110.1,40.9,89.2,45.9,11.2,32.5,...,24.5,7.8,4.5,12.5,23.2,101.4,51.9,106.6,108.3,0
